<a href="https://colab.research.google.com/github/akram01Br/PixelHawk-OD/blob/main/FilmsPopulairesDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession,Row

In [ ]:
spark =SparkSession.builder.config("spark.sql.warehouse.dir","file:///home/cj/sparkWorkSpace/temp").appName("FilmsPopulairesDF").getOrCreate()

21/12/08 13:54:42 WARN Utils: Your hostname, cj-X550JX resolves to a loopback address: 127.0.1.1; using 172.16.50.1 instead (on interface wlp3s0)
21/12/08 13:54:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/12/08 13:54:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [ ]:
lines=spark.sparkContext.textFile("file:///home/cj/sparkWorkSpace/DataBase/u.data")

In [ ]:
def convertRow(line,movies):
    fields=line.split(',')
    return Row(ID=int(fields[0]))

In [ ]:
def loadNamesFilms():
    moviesNames={}
    with open("DataBase/u.item",encoding = "ISO-8859-1") as f:
        for line in f:
            fields=line.split("|")
            moviesNames[int(fields[0])]=fields[1]
        return moviesNames

In [ ]:
nameDict=loadNamesFilms()

In [ ]:
lines=lines.map(lambda x:Row(movieID=int(x.split()[1])))


In [ ]:
#creating the dataFrame
MovieDataFrame=spark.createDataFrame(lines).cache()
MovieDataFrame.createOrReplaceTempView("film")


In [ ]:
topMovies=MovieDataFrame.groupby("movieID").count().orderBy("count",ascending=False)

In [ ]:
topMovies.show()

+-------+-----+
|movieID|count|
+-------+-----+
|     50|  583|
|    258|  509|
|    100|  508|
|    181|  507|
|    294|  485|
|    286|  481|
|    288|  478|
|      1|  452|
|    300|  431|
|    121|  429|
|    174|  420|
|    127|  413|
|     56|  394|
|      7|  392|
|     98|  390|
|    237|  384|
|    117|  378|
|    172|  367|
|    222|  365|
|    313|  350|
+-------+-----+
only showing top 20 rows



In [ ]:
top10films=topMovies.take(10)
for film in top10films:
    print("%s : %d"% (nameDict[film[0]],film[1]))

Star Wars (1977) : 583
Contact (1997) : 509
Fargo (1996) : 508
Return of the Jedi (1983) : 507
Liar Liar (1997) : 485
English Patient, The (1996) : 481
Scream (1996) : 478
Toy Story (1995) : 452
Air Force One (1997) : 431
Independence Day (ID4) (1996) : 429
